In [ ]:
import yt
import os
import numpy as np
import glob
import re

# Parameters
alpha0 = 4 #2.05 #1.3 #1.5 #0.5 #1.5
temperature = 0. #5e-5 #2e-6 #0. #1e-5
nx, ny, nz = 64, 64, 64 #32, 32, 32
radius = 0.22 #0.425
rho_lo = 0.0
rho_hi = 1. #0.7 #3.0
step1 = 0
step2 = 30000 #10000
plot_int = 400 #50 #400

# Construct file paths
data_root = "./"
#data_root = "./critical_param_test_no_pseudo/"
#data_root = "./critical_param_test_pseudo/"
#data_root = "./droplet_surface_tension/"
#data_root = "./nucleation_test/"
data_dir = data_root + "data_droplet_density_{:.2f}_alpha0_{:.2f}_r{:.3f}_size{:d}-{:d}-{:d}/".format(rho_hi, alpha0, radius, nx, ny, nz)

data_subdir = ""
if temperature > 0:
    data_subdir = "lbm_data_shshan_alpha0_{:.2f}_xi_{:.1e}_size{:d}-{:d}-{:d}_continue/".format(alpha0, temperature, nx, ny, nz)
else:
    data_subdir = "lbm_data_shshan_alpha0_{:.2f}_xi_{:.1e}_size{:d}-{:d}-{:d}/".format(alpha0, temperature, nx, ny, nz)

IMG_folder = data_dir + "IMG_data_shshan_alpha0_{:.2f}_xi_{:.1e}_size{:d}-{:d}-{:d}/".format(alpha0, temperature, nx, ny, nz)

# Create image directory if it doesn't exist
os.makedirs(IMG_folder, exist_ok=True)
all_plt_files = sorted(glob.glob(os.path.join(data_dir+data_subdir, "plt*")))

# Extract step numbers from filenames (assuming format like plt00010, plt12345, etc.)
def extract_step(filepath):
    basename = os.path.basename(filepath) # Get the file/folder name from the path
    match = re.search(r'plt(\d+)', basename)  # It finds single digits with \d and sequences of digits with \d+.
    if match:
        return int(match.group(1)) # Extract the digits in filename and convert to integer
    else:
        return -1  # invalid
# Load all plot files (e.g., plt0000, plt0001, ...)
ts = yt.load(data_dir+data_subdir + "plt*")

# Filter files within [step1, step2] and divisible by plot_int (or aligned to step1 + n*plot_int)
selected_files = []
for f in all_plt_files:
    step = extract_step(f)
    if step == -1:
        continue
    if step1 <= step <= step2 and (step - step1) % plot_int == 0:
        selected_files.append((step, f))

# Sort by step number
selected_files.sort(key=lambda x: x[0])

# Process each selected file
ac = 0
for step, filepath in selected_files:
    fr = yt.load(filepath)
    slc = yt.SlicePlot(fr, "z", ("boxlib", "rho"))
    slc.set_log(("boxlib", "rho"), False)
    # Optional: set color limits
    # slc.set_zlim(("boxlib", "rho"), rho_lo - 0.3, rho_hi + 0.5)
    
    filename = os.path.join(IMG_folder, f"fr{ac:07d}.png")
    slc.save(filename)
    ac += 1

print(f"Saved {ac} frames from step {step1} to {step2} (interval={plot_int})")

yt : [INFO     ] 2025-12-24 07:38:08,210 Parameters: current_time              = 0.0
yt : [INFO     ] 2025-12-24 07:38:08,210 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2025-12-24 07:38:08,211 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-12-24 07:38:08,248 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-12-24 07:38:09,655 xlim = 0.000000 1.000000
yt : [INFO     ] 2025-12-24 07:38:09,655 ylim = 0.000000 1.000000
yt : [INFO     ] 2025-12-24 07:38:09,656 xlim = 0.000000 1.000000
yt : [INFO     ] 2025-12-24 07:38:09,657 ylim = 0.000000 1.000000
yt : [INFO     ] 2025-12-24 07:38:09,660 Making a fixed resolution buffer of (('boxlib', 'rho')) 800 by 800
yt : [INFO     ] 2025-12-24 07:38:12,538 Saving plot ./data_droplet_density_1.00_alpha0_4.00_r0.220_size64-64-64/IMG_data_shshan_alpha0_4.00_xi_0.0e+00_size64-64-64/fr0000000.png
yt : [INFO     ] 2025-12-24 07:38:13,424 Parameters: current_time              = 50.0
yt

KeyboardInterrupt: 

In [4]:
import imageio
import re
from pathlib import Path
import os

# Parameters
alpha0 = 4. #2.5 #3.0 #1.3 #1.5 #0.5 #1.5
temperature = 5e-5 #1e-5 #2e-6 #0. #1e-5
nx, ny, nz = 64, 64, 64 #32, 32, 32
radius = 0.2
rho_lo = 0.0
rho_hi = 1. #3.0

# Construct file paths
data_root = "./"
#data_root = "./critical_param_test_no_pseudo/"
#data_root = "./droplet_surface_tension/"
#data_root = "./nucleation_test/"
data_dir = data_root + "data_droplet_density_{:.2f}_alpha0_{:.2f}_r{:.3f}_size{:d}-{:d}-{:d}/".format(rho_hi, alpha0, radius, nx, ny, nz)

data_subdir = ""
if temperature > 0:
    data_subdir = "lbm_data_shshan_alpha0_{:.2f}_xi_{:.1e}_size{:d}-{:d}-{:d}_continue/".format(alpha0, temperature, nx, ny, nz)
else:
    data_subdir = "lbm_data_shshan_alpha0_{:.2f}_xi_{:.1e}_size{:d}-{:d}-{:d}/".format(alpha0, temperature, nx, ny, nz)

IMG_folder = data_dir + "IMG_data_shshan_alpha0_{:.2f}_xi_{:.1e}_size{:d}-{:d}-{:d}/".format(alpha0, temperature, nx, ny, nz)


folder = Path(IMG_folder)     
output = IMG_folder + "movie.mp4"  
os.makedirs("out", exist_ok=True)
print("Output file:", output)    
fps = 30             

files = []
for f in folder.iterdir():
    m = re.match(r"fr(\d{7})\.(png|jpg|jpeg)", f.name, re.IGNORECASE)
    if m:
        files.append((int(m.group(1)), f))

files = [f for _, f in sorted(files)]

writer = imageio.get_writer(output, fps=fps)
for f in files:
    writer.append_data(imageio.imread(f))
writer.close()

print("Saved to", output)


Output file: ./data_droplet_density_1.00_alpha0_4.00_r0.200_size64-64-64/IMG_data_shshan_alpha0_4.00_xi_5.0e-05_size64-64-64/movie.mp4


/tmp/ipykernel_3000266/603444877.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(f))
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1085, 920) to (1088, 928) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saved to ./data_droplet_density_1.00_alpha0_4.00_r0.200_size64-64-64/IMG_data_shshan_alpha0_4.00_xi_5.0e-05_size64-64-64/movie.mp4
